## PHAGES2050
**Date:** 24.08.2020  
**Author:** Pior Tynecki

That notebook is a part of examples in the [PHAGES2050](https://github.com/ptynecki/PHAGES2050) repository. The content of it is valid and was tested with 0.0.4 framework version.

Two belows examples are presenting the embedding model execution used for protein vectorization. The vectorization is producing 1024 feature space for each protein or for the set of proteins representing single bacteriophage.
Depends on your needs you can use fixed-length numeric vectors for:
* single protein function prediction,
* feature space creation for further Machine Learning classifications,
* feature space creation for bacteriophages deeper exploration in 3D;

### Import necessary modules

In [1]:
import torch
import pandas as pd

In [2]:
from phages2050.embeddings.proteins.bert import BertModelManager, BertEmbedding

### Download pre-trained proteins model
Before you start vectorizing protein sequence or set of proteins you have to download pre-trained model dedicated for proteins embedding.  

This step have to be done once and doesn't have to be repeated on single environment.

In [3]:
bert_model_manager = BertModelManager()

In [4]:
# It can take a while and depends on the Internet speed
# Be aware that model size is 1.4 GB
bert_model_dir = bert_model_manager.download_model()

# If succeed expect pre-trained model directory path
bert_model_dir

'bert_model/bert/'

### Preapre DataFrame

PHAGES2050 standard I/O data format is a pandas DataFrame.

Put your protein sequence into DataFrame with their known function name or left it as "Unknown" value.

In [5]:
# Non-fixed length protein string made up of 20 different amino acids
protein_sequence = "MSMYTTAQLLAANEQKFKFDPLFLRLFFRESYPFTTEKVYLSQIPGLVNMALYVSPIVSGEVIRSRGGSTSEFTPGYVKPKHEVNP" \
                   "QMTLRRLPDEDPQNLADPAYRRRRIIMQNMRDEELAIAQVEEMQAVSAVLKGKYTMTGEAFDPVEVDMGRSEENNITQSGGTEWSK" \
                   "RDKSTYDPTDDIEAYALNASGVVNIIVFDPKGWALFRSFKAVKEKLDTRRGSNSELETAVKDLGKAVSYKGMYGDVAIVVYSGQYV" \
                   "ENGVKKNFLPDNTMVLGNTQARGLRTYGCIQDADAQREGINASARYPKNWVTTGDPAREFTMIQSAPLMLLADPDEFVSVQLA"

# Protein function name
protein_class = "Unknown"

# Group it as list of two elements
protein_data = [
    protein_sequence,
    protein_class
]

In [6]:
# Create temporary DataFrame
df = pd.DataFrame(
    data=[protein_data],
    columns=['sequence', 'class']
)
df.head()

,sequence,class
0,MSMYTTAQLLAANEQKFKFDPLFLRLFFRESYPFTTEKVYLSQIPG...,Unknown


### Check CUDA and GPU support
If you have CUDA and GPU support then each of the calculation below will be executed much more faster than with CPU.

In [7]:
# Check if you have CUDA support
torch.cuda.is_available()

True

In [8]:
# Check which GPU has expected ID
gpu_device_count = torch.cuda.device_count()
{gpu_id: torch.cuda.get_device_name(gpu_id) for gpu_id in range(gpu_device_count)}

{0: 'TITAN V', 1: 'GeForce RTX 2070'}

### Single protein vectorization - example 1
This example shows how to vectorize single protein sequence into 1024 numeric vector space.

In [9]:
bert_embedding = BertEmbedding(
    model_dir=bert_model_dir,
    # Pass expected GPU ID
    cuda_device=0
)

In [10]:
embbeding_result = bert_embedding.transform(df=df)

In [11]:
embbeding_result.head()

,BERT_0,BERT_1,BERT_2,BERT_3,BERT_4,BERT_5,BERT_6,BERT_7,BERT_8,BERT_9,...,BERT_1015,BERT_1016,BERT_1017,BERT_1018,BERT_1019,BERT_1020,BERT_1021,BERT_1022,BERT_1023,class
0,-0.00758,-0.008452,-0.013748,0.022192,0.072757,-0.020868,0.002841,0.030178,-0.028072,0.038762,...,-0.030086,0.005809,0.002216,-0.039288,0.008029,0.003874,0.023775,-0.002863,0.01145,Unknown


### Bacteriophage averaged vectorization - example 2
This example shows how to vectorize set of bacteriophage proteins sequences into 1024 (averaged) numeric vector space.

In [12]:
# Non-fixed length protein string made up of 20 different amino acids
protein_sequence_1 = "MSMYTTAQLLAANEQKFKFDPLFLRLFFRESYPFTTEKVYLSQIPGLVNMALYVSPIVSGEVIRSRGGSTSEFTPGYVKPKHEVNP" \
                     "QMTLRRLPDEDPQNLADPAYRRRRIIMQNMRDEELAIAQVEEMQAVSAVLKGKYTMTGEAFDPVEVDMGRSEENNITQSGGTEWSK" \
                     "RDKSTYDPTDDIEAYALNASGVVNIIVFDPKGWALFRSFKAVKEKLDTRRGSNSELETAVKDLGKAVSYKGMYGDVAIVVYSGQYV" \
                     "ENGVKKNFLPDNTMVLGNTQARGLRTYGCIQDADAQREGINASARYPKNWVTTGDPAREFTMIQSAPLMLLADPDEFVSVQLA"

protein_id_1 = "NP_040587.1"

protein_sequence_2 = "MNLKEQRKAKMAEALNFKARIKAGDALSDDEVKALTALLGDIESLDAQIAKAEENAKVLASVAQLPDADSEKPTGTDGGAKASSLG" \
                     "AHFIKELKASGRSLKDPGSFVSTEFKAATDTQSVGSAGGAFGPLITDVDRTPVIPFQRPLAVADLMASGSVSGNAITYPVFGALEG" \
                     "TSTFVAEGAAKPQLHVANPEWKTDALGEIAAWFKVTDDMAEDIPYLVSEIQSTAIYDLALREELALLSGDGSASSITGILNRPGVQ" \
                     "TIAKGTDSDPDRIFSAISKVQEVTGFVADGIVINPADYQAIRLSKDANGQYFGGGFFSGQYGTGGIVQNPPLWGLKTVVTSSIAQG" \
                     "TTVVGPFKAAAKVFRKGGLRVESTNSHADDFTNDKITIRIRERLGLQVKYPSAFVKVALGTAGK"


protein_id_2 = "YP_009596870.1"

# Group it as list of lists with two elements each
bacteriophage_data = [
    [protein_sequence_1, protein_id_1],
    [protein_sequence_2, protein_id_2]
]

In [13]:
# Create temporary DataFrame
df_phage = pd.DataFrame(
    data=bacteriophage_data,
    columns=['sequence', 'name']
)
df_phage.head()

,sequence,name
0,MSMYTTAQLLAANEQKFKFDPLFLRLFFRESYPFTTEKVYLSQIPG...,NP_040587.1
1,MNLKEQRKAKMAEALNFKARIKAGDALSDDEVKALTALLGDIESLD...,YP_009596870.1


In [16]:
embbeding_result = bert_embedding.transform(df=df_phage, bacteriophage_level=True)

[DEBUG] Protein vectors are averaging to form a bacteriophage


In [15]:
embbeding_result.head()

,BERT_0,BERT_1,BERT_2,BERT_3,BERT_4,BERT_5,BERT_6,BERT_7,BERT_8,BERT_9,...,BERT_1015,BERT_1016,BERT_1017,BERT_1018,BERT_1019,BERT_1020,BERT_1021,BERT_1022,BERT_1023,name
0,-0.006764,-0.01259,-0.014861,0.016062,0.06595,-0.019135,0.006641,0.01201,-0.030033,0.024458,...,-0.02204,0.005218,-0.00583,-0.010436,0.005666,0.011336,0.019985,-0.011375,0.024778,NP_040587.1
